# Star Wars Rebels (DVD)

<https://thetvdb.com/series/star-wars-rebels>

![Star Wars Rebels Clearart](https://artworks.thetvdb.com/banners/v4/series/283468/clearart/611b7d0eb3bea.png)


These notes preserve my memories of auto-generating [Kodi](https://kodi.tv/) [[GitHub](https://github.com/xbmc)] `tvshow` and `episodedetails` XML files in `*.nfo` formats.

This `*.nfo` format is kind of covered in the following documents:

- [NFO files](https://kodi.wiki/view/NFO_files)
- [NFO files/TV shows](https://kodi.wiki/view/NFO_files/TV_shows)
- [NFO files/Episodes](https://kodi.wiki/view/NFO_files/Episodes)


In [48]:
%run beautifulsoup-thetvdb.py

from IPython.core.display import display, HTML

dvd_episode_location = 'https://thetvdb.com/series/star-wars-rebels/seasons/dvd/1'

soup = getTheTVDBSoup(dvd_episode_location)
table = soup.find('table')

display(HTML(table.decode()))

,Name,First Aired,Runtime,Image,Certified
S01E01,Spark of Rebellion (1),"October 3, 2014 Disney XD",25,,
S01E02,Spark of Rebellion (2),"October 3, 2014 Disney XD",25,,
S01E03,Droids in Distress,"October 13, 2014 Disney XD",25,,
S01E04,Fighter Flight,"October 20, 2014 Disney XD",25,,
S01E05,Rise of the Old Masters,"October 27, 2014 Disney XD",25,,
S01E06,Breaking Ranks,"November 3, 2014 Disney XD",25,,
S01E07,Out of Darkness,"November 10, 2014 Disney XD",25,,
S01E08,Empire Day,"November 17, 2014 Disney XD",25,,
S01E09,Gathering Forces,"November 24, 2014 Disney XD",25,,
S01E10,Path of the Jedi,"January 5, 2015 Disney XD",25,,


In [45]:
episode_data = list(yieldEpisodeData(table))
episode_data

[{'episode': 'S01E01',
  'title': 'Spark of Rebellion (1)',
  'plotUri': '/series/star-wars-rebels/episodes/7460333',
  'plot': 'It is a dark time. The Jedi are no more, and the Empire rules the galaxy with an iron fist -- bringing tyranny to the Outer Rim world of Lothal. Their ominous presence is a cloud over the planet, squashing any hope of freedom or a better life, especially for 14-year-old orphan Ezra Bridger. But the young con artist soon finds that destiny has other plans for him, as he encounters a small band of rebels who dare to strike back against the Empire.'},
 {'episode': 'S01E02',
  'title': 'Spark of Rebellion (2)',
  'plotUri': '/series/star-wars-rebels/episodes/7460334',
  'plot': 'Ezra and the crew learn that Wookiee slaves are being taken by the Empire to the spice mines of Kessel and launch a mission to rescue them, all while being pursued by Agent Kallus of the Imperial Security Bureau.'},
 {'episode': 'S01E03',
  'title': 'Droids in Distress',
  'plotUri': '/se

## using `episode_data` to generate `*.nfo` files


In [46]:
nfo_location_template = './beautifulsoup-star-wars-rebels/Star Wars Rebels (2014)'

In [49]:
for item in episode_data:
    current_episode = item['episode']
    current_xml_tree = getXmlTree(item['title'], item['plot'])
    writeXml(nfo_location_template, current_episode, current_xml_tree)
